In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, models

from torch.autograd import Variable
import torch.utils.data as data
import pandas as pd
import os
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset

import sklearn.model_selection as sk



In [3]:
# Path for (unlabeled) test data
test_path = '/content/drive/MyDrive/FashionClassification/Test/'

In [4]:
import re
def num_sort(test_string):
   return list(map(int, re.findall(r'\d+', test_string)))[0]
 

In [5]:
# Prepare test data set

data=[]
label=[]

files = os.listdir(test_path)
files.sort(key=num_sort)

for f in files:
  img = Image.open(test_path+f)
  one_img = np.asarray(np.float32(img))
  norm_img = (one_img-np.min(one_img))/(np.max(one_img)-np.min(one_img))
  data.append(norm_img.transpose(2,0,1))  # (150,150,3) --> (3,150,150)
  label.append(0)    # dummy label for preparing dataloader

        
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

test_X = torch.from_numpy(data)
test_Y = torch.from_numpy(label)

test_X = test_X.type(torch.cuda.FloatTensor)
test_Y = test_Y.type(torch.cuda.LongTensor)

test_dataset = TensorDataset(test_X,test_Y)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [6]:
print(test_X.shape, test_Y.shape)

torch.Size([6, 3, 150, 150]) torch.Size([6])


In [7]:
# VGG type dict
# int : output chnnels after conv layer
# 'M' : max pooling layer
VGG_types = {
    'VGG16' : [64,64, 'M', 128, 128, 'M', 256, 256,256, 'M', 512,512,512, 'M',512,512,512,'M'],
    'VGG19' : [64,64, 'M', 128, 128, 'M', 256, 256,256,256, 'M', 512,512,512,512, 'M',512,512,512,512,'M']
}

# define VGGnet class
class VGGnet(nn.Module):
    def __init__(self, model, in_channels=3, num_classes=3, init_weights=True):
        super(VGGnet,self).__init__()
        self.in_channels = in_channels

        # create conv_layers corresponding to VGG type
        self.conv_layers = self.create_conv_laters(VGG_types[model])

        self.fcs = nn.Sequential(
            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(64, num_classes),

        )

        # weight initialization
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0),-1)
        x = self.fcs(x)
        return x

    # defint weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    # define a function to create conv layer taken the key of VGG_type dict 
    def create_conv_laters(self, architecture):
        layers = []
        in_channels = self.in_channels # 3

        for x in architecture:
            if type(x) == int: # int means conv layer
                out_channels = x

                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                     kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                           nn.BatchNorm2d(x),
                           nn.ReLU()]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
        
        return nn.Sequential(*layers)


# creat VGGnet object
model = VGGnet('VGG16', in_channels=3, num_classes=3, init_weights=True)
model.cuda()

In [8]:
# load trained model
model = torch.load('/content/drive/MyDrive/FashionClassification/Model_TeamC.pt')

In [9]:
# evaluation
model.eval()
f_pred = []

with torch.no_grad():
  for inputs, labels in test_loader:
    inputs, labels = Variable(inputs), Variable(labels)
    output = model(inputs)

    idx, pred = output.max(1)
    pred = (pred.detach().cpu().numpy())

    f_pred.extend(pred.tolist())

In [10]:
# converting 0/1/2 label value into Resort/Retro/Sporty

for index, value in enumerate(f_pred):
  if value==0:
    f_pred[index]='Resort'
  elif value==1:
    f_pred[index]='Retro'
  else:
    f_pred[index]='Sporty'

In [11]:
# Submission file
img_files = [i for i in range(len(f_pred))]
submission = pd.DataFrame(index = range(0,len(f_pred)), columns = {'img','class'})
submission['img'] = img_files
submission['class'] = f_pred
submission.to_csv('/content/drive/MyDrive/FashionClassification/submission.csv', index=False)

In [12]:
submission

,class,img
0,Sporty,0
1,Sporty,1
2,Sporty,2
3,Sporty,3
4,Sporty,4
5,Sporty,5
